In [ ]:
!pip install opencv-python==4.7.0.72
!pip install speechrecognition==3.10.4
!pip install threading
!pip install google-cloud-translate==3.15.3
!pip install pyttsx3==2.90
!pip install gtts==2.5.1
!pip install playsound==1.3.0

ERROR: Could not find a version that satisfies the requirement threading (from versions: none)
ERROR: No matching distribution found for threading


In [ ]:
import os
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\USER\Downloads\youtubeandroid-numberofID.json"

print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

In [ ]:
import speech_recognition as sr
import gtts
import playsound
from google.cloud import translate_v2 as translate


#for translation
def translate_text(text, target_language='en'):
    client = translate.Client()
    result = client.translate(text, target_language=target_language)
    translated_text = result['translatedText']
    return translated_text

#for microphpone
recognizer = sr.Recognizer()

#pause threshold = for optimization
recognizer.pause_threshold = 0.5
while True:
    with sr.Microphone() as source:
        print("Adjusting for ambient noise, please wait...")
        recognizer.adjust_for_ambient_noise(source, duration=1)

        print("Speak Now")
        try:
            voice = recognizer.listen(source)
            text = recognizer.recognize_google(voice, language="az")
            print("Original Text:", text)


            text_en = translate_text(text, target_language='en')
            print("Translated Text:", text_en)


            converted_audio = gtts.gTTS(text_en, lang="en")
            p = "C:\\Users\\USER\\Project\\"
            converted_audio.save(p + "hello.mp3")


            playsound.playsound(p + "hello.mp3")
        except sr.WaitTimeoutError:
            print("Listening timed out while waiting for phrase to start")
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))

Adjusting for ambient noise, please wait...
Speak Now
Original Text: salam
Translated Text: hello


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\USER\\Project\\hello.mp3'

In [ ]:
import cv2
import speech_recognition as sr
import threading
from google.cloud import translate_v2 as translate
import pyttsx3

# Initialize recognizer and microphone for speech recognition
recognizer = sr.Recognizer()
microphone = sr.Microphone()

# Initialize Google Cloud Translate client and pyttsx3 text-to-speech engine
translate_client = translate.Client()
engine = pyttsx3.init()

# Global variable to store the translated transcript
transcript = ""


def translate_text(text, target_language='en'):
    """
    Translates text to the target language.
    """
    result = translate_client.translate(text, target_language=target_language)
    return result['translatedText']


def text_to_speech(text):
    """
    Converts text to speech.
    """
    engine.say(text)
    engine.runAndWait()

# Set recognizer parameters
recognizer.pause_threshold = 1


def update_transcript():
    """
    Listens to the microphone and updates the transcript.
    """
    global transcript

    while True:
        with microphone as source:
            recognizer.adjust_for_ambient_noise(source, duration=1)   # Adjust microphone for ambient noise

            try:
                print("Listening...")
                audio = recognizer.listen(source)
                print("Processing...")
                original_text = recognizer.recognize_google(audio, language="az")  # Recognize speech using Google Web Speech API
                print("Original Text: ", original_text)


                translated_text = translate_text(original_text, 'en')   # Translate recognized text
                print("Translated Text: ", translated_text)

                transcript = translated_text     # Update the transcript

                # Convert translated text to speech
                text_to_speech(transcript)
            except sr.WaitTimeoutError:
                print("Timeout; no speech detected.")
                transcript = "Timeout; no speech detected."
            except sr.UnknownValueError:
                print("Could not understand the audio.")
                transcript = "Could not understand the audio."
            except sr.RequestError as e:
                print(f"Could not request results; {e}")
                transcript = f"Service error: {e}"

# Start the transcript update function in a daemon thread
thread = threading.Thread(target=update_transcript)
thread.daemon = True
thread.start()

# Open a connection to the webcam
cap = cv2.VideoCapture(0)


while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture video")
        break

    # Get the frame dimensions
    height, width, _ = frame.shape

    # Draw a white rectangle at the bottom of the frame for displaying the transcript
    rect_height = 80
    cv2.rectangle(frame, (0, height - rect_height), (width, height), (255, 255, 255), -1)

    # Define the position for the transcript text
    text_position = (10, height - 40)
    cv2.putText(frame, transcript, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)

    # Display the video feed with the transcript
    cv2.imshow('Real-Time Video Feed with Transcript', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


Listening...
Processing...
Original Text:  salam
Translated Text:  hello
Listening...


Processing...
Original Text:  çox sağ olun
Translated Text:  Thank you very much
